In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [3]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [5]:
if True:
  from unsloth import FastLanguageModel
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = "/content/drive/MyDrive/LexBot/LexBot", # YOUR MODEL YOU USED FOR TRAINING
      max_seq_length = 2048,
      dtype = None,
      load_in_4bit = True,
  )
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given the following conversation, relevant context, and a follow up question, reply with an answer to the current question the user is asking. Return only your response to the question given the above information following the users instructions as needed. Explain elobrately accordind to indan penal code or articales, make the response big with detailed explanation.", # instruction
        "What is the maximum penalty for drink and drive", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)
print()

In [ ]:
import torch
from transformers import TextStreamer
device = "cuda" if torch.cuda.is_available() else "cpu"
conversation_history = []
text_streamer = TextStreamer(tokenizer)

while True:
    question = input("Enter your question: ")
    conversation_history.append(f"User: {question}")
    history_text = "\n".join(conversation_history)

    prompt = alpaca_prompt.format(
        "Given the following conversation, relevant context, and a follow-up question, reply with an answer to the current question the user is asking. Return only your response following the user's instructions as needed. Explain elaborately according to the Indian Penal Code or relevant articles, providing a detailed and extensive explanation.",
        f"Conversation so far:\n{history_text}\n\nExplain in the context of INDIAN LAW: {question}",
        ""  # Leave this blank for generation
    )

    inputs = tokenizer([prompt], return_tensors="pt").to(device)
    output_tokens = model.generate(**inputs, max_new_tokens=512)

    response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    response = response.split("### Response:\n")[-1].strip()

    conversation_history.append(f"Assistant: {response}")

    # Print only the AI's response
    print("\nAI:", response)
    print("\n")  # Extra spacing for readability
